In [1]:
import sympy as sp
sp.init_printing(use_unicode=True)
from sympy.abc import a,k,x


from sympy.codegen.ast import Assignment
from sympy.codegen.ast import real, float80
from sympy.utilities.codegen import codegen

from sympy.algebras.quaternion import Quaternion 

# Highest level

## Generate C-code

In [2]:
expr = (sp.Rational(-1, 2) * x * k * (a**2) )

sp.ccode(expr)

'-1.0/2.0*pow(a, 2)*k*x'

In [3]:
sp.ccode(expr, assign_to="E", type_aliases={real: float80})

'E = -1.0L/2.0L*powl(a, 2)*k*x;'

In [4]:
length, breadth, height = sp.symbols('length, breadth, height')

expr = length*breadth*height

[(c_name, c_code), (h_name, c_header)] = \
codegen(('volume', expr), "C99", "test", header=False, empty=False)

print(c_code)

#include "test.h"
#include <math.h>
double volume(double breadth, double height, double length) {
   double volume_result;
   volume_result = breadth*height*length;
   return volume_result;
}



## Generate julia code

In [5]:
print(sp.julia_code(expr, assign_to="H_is"))

H_is = breadth .* height .* length


## Lambdify

In [19]:
from sympy.utilities.lambdify import implemented_function
from sympy import lambdify

f = implemented_function('f', lambda x: Quaternion(x**2, x**3, x))
lam_f = lambdify(x, f(x))
lam_f(4)

# Intermediate level

# Demos

### Rotation matrix to quaternion

In [26]:
Rx = sp.rot_axis3(x)

q = sp.trigsimp(Quaternion.from_rotation_matrix(Rx))

q1 = q.a
q2 = q.a
q3 = q.a
q4 = q.a

expr = q.a

[(c_name, c_code), (h_name, c_header)] = \
codegen(('quat', expr), "C99", "test", header=False, empty=False)
print(c_code)

#include "test.h"
#include <math.h>
double quat(double x) {
   double quat_result;
   quat_result = (1.0/2.0)*M_SQRT2*sqrt(cos(x) + 1);
   return quat_result;
}



In [27]:
# # write to file
# codegen(('quat', expr), "C99", "test", header=False, empty=False,  to_files=True)

# print(c_code)
# !gcc test.c -lm -o run

###  quaternion to Rotation matrix

In [37]:
qw, qx, qy, qz = sp.symbols("qw qx qy qz") 
q = Quaternion(qw, qx, qy, qz)

R = sp.trigsimp(q.to_rotation_matrix())
expr = R

# X = Matrix([[x],[y],[z]])
# expr = R*X

[(c_name, c_code), (h_name, c_header)] = \
codegen(('quat', expr), "C99", "test", header=False, empty=False)
print(c_code)

#include "test.h"
#include <math.h>
void quat(double qw, double qx, double qy, double qz, double *out_2346411308148986310) {
   out_2346411308148986310[0] = -2*(pow(qy, 2) + pow(qz, 2))/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2)) + 1;
   out_2346411308148986310[1] = 2*(-qw*qz + qx*qy)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_2346411308148986310[2] = 2*(qw*qy + qx*qz)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_2346411308148986310[3] = 2*(qw*qz + qx*qy)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_2346411308148986310[4] = -2*(pow(qx, 2) + pow(qz, 2))/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2)) + 1;
   out_2346411308148986310[5] = 2*(-qw*qx + qy*qz)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_2346411308148986310[6] = 2*(-qw*qy + qx*qz)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_2346411308148986310[7] = 2*(qw*qx + qy*qz)/(pow(qw, 2) + pow(qx, 2) + pow(qy, 2) + pow(qz, 2));
   out_23464113081

## quaternion multiplication

In [38]:
qw, qx, qy, qz = sp.symbols("qw qx qy qz") 
q = Quaternion(qw, qx, qy, qz)

q_result = q.mul(q)

expr = q_result.a

# X = Matrix([[x],[y],[z]])
# expr = R*X

[(c_name, c_code), (h_name, c_header)] = \
codegen(('quat', expr), "C99", "test", header=False, empty=False)
print(c_code)

#include "test.h"
#include <math.h>
double quat(double qw, double qx, double qy, double qz) {
   double quat_result;
   quat_result = pow(qw, 2) - pow(qx, 2) - pow(qy, 2) - pow(qz, 2);
   return quat_result;
}



## quaternion exponential

In [13]:
qw, qx, qy, qz = sp.symbols("qw qx qy qz") 
q = Quaternion(qw, qx, qy, qz)

expr = q.exp()

print(sp.julia_code(expr.d, assign_to="q_exp"))

q_exp = qz .* exp(qw) .* sin(sqrt(qx .^ 2 + qy .^ 2 + qz .^ 2)) ./ sqrt(qx .^ 2 + qy .^ 2 + qz .^ 2)
